In [49]:
# 0514 고박사님과 함께
import numpy as np

#아핀변환

class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        
        self.x = None
        self.original_x_shape = None
        # 가중치와 편향 매개변수의 미분
        self.dW = None
        self.db = None

    def forward(self, x):
        # 텐서 대응
        self.original_x_shape = x.shape
        x = x.reshape(x.shape[0], -1)
        self.x = x

        out = np.dot(self.x, self.W) + self.b

        return out

    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        
        dx = dx.reshape(*self.original_x_shape)  # 입력 데이터 모양 변경(텐서 대응)
        return dx

class Sigmoid :
    def __init__(self):
        self.x = None
        self.y = None
    def forward(self, y):
        self.x = y
        self.y = 1 / (1 + np.exp(-y))
        return self.y
    
    def backward(self, dout) :
        dx = dout * (1.0 - self.y) * self.y
        return dx

#Mse
class MeanSquaredError:
    def __init__(self):
        self.loss = None  # 손실 값
        self.y = None     # 예측값
        self.t = None     # 정답값

    def forward(self, y, t):
        self.y = y
        self.t = t
        # 오차 계산
        self.loss = 0.5 * np.mean((self.y - self.t) ** 2)
        return self.loss

    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) * dout / batch_size
        return dx

In [50]:
z = np.array([1, 1])
z = np.reshape(z, (1, -1))
print(z)
print(np.shape(z))

t = np.array([0])
print(t)

[[1 1]]
(1, 2)
[0]


In [51]:
W = np.random.randn(2, 1)
b = np.zeros(1,)
affine = Affine(W,b)
mse = MeanSquaredError()

In [52]:
out = affine.forward(z)
print(out)
out = mse.forward(out, t)
print("손실값 : ", out)

[[1.04780192]]
손실값 :  0.5489444300198707


In [53]:
dout = np.ones(1)
dout = mse.backward(dout)
print("mse 미분 : ", dout)
dout = affine.backward(dout)
print("dx : ", dout)

mse 미분 :  [[1.04780192]]
dx :  [[0.61527638 0.48261248]]


In [54]:
print(affine.dW)
print(affine.db)


[[1.04780192]
 [1.04780192]]
[1.04780192]


In [55]:
z = np.array([1,1])
z = np.reshape(z, (1, -1))
t = np.array([1])
print(z)
print(t)

[[1 1]]
[1]


In [56]:
W1 = np.random.randn(2,2)
b1 = np.zeros(2,)
affine1 = Affine(W1, b1)

W2 = np.random.randn(2,1)
b2 = np.zeros(1,)
affine2 = Affine(W2, b2)

print("1번째 선형변환 가중치 행렬 : \n", affine1.W)
print("1번째 선형변환 절편 벡터 : \n", affine1.b)

print("2번째 선형변환 가중치 행렬 : \n", affine2.W)
print("2번째 선형변환 절편 벡터 : \n", affine2.b)

mse = MeanSquaredError()

1번째 선형변환 가중치 행렬 : 
 [[-0.06368479  0.7776441 ]
 [-0.925074    0.29076634]]
1번째 선형변환 절편 벡터 : 
 [0. 0.]
2번째 선형변환 가중치 행렬 : 
 [[-0.5003567 ]
 [ 0.88822739]]
2번째 선형변환 절편 벡터 : 
 [0.]


In [57]:
y1 = affine1.forward(z)
print("1번째 선형변환 결과 : \n", y1)
print("1번째 선형변환 결과 모양 : \n", np.shape(y1))

1번째 선형변환 결과 : 
 [[-0.98875879  1.06841044]]
1번째 선형변환 결과 모양 : 
 (1, 2)


In [58]:
y2 = affine2.forward(y1)
print("2번째 선형변환 결과 : \n", y2)
print("2번째 선형변환 결과 모양 : \n", np.shape(y2))

2번째 선형변환 결과 : 
 [[1.4437235]]
2번째 선형변환 결과 모양 : 
 (1, 1)


In [59]:
loss = mse.forward(y2, t)
print("손실값 : ", loss)

손실값 :  0.09844527267050193


In [61]:
dout = np.ones(1,)
dout = mse.backward(dout)
print("mse 에 대한 미분 : ", dout)

mse 에 대한 미분 :  [[0.4437235]]


In [62]:
dout = affine2.backward(dout)
print("2번째 선형변환에 대한 미분 : ", dout)

2번째 선형변환에 대한 미분 :  [[-0.22202003  0.39412737]]


In [63]:
print(affine2.dW)
print(affine2.db)

[[-0.43873551]
 [ 0.47407882]]
[0.4437235]


In [64]:
dout = affine1.backward(dout)
print(affine1.dW)
print(affine1.db)

[[-0.22202003  0.39412737]
 [-0.22202003  0.39412737]]
[-0.22202003  0.39412737]


In [69]:
lr = 0.01
for i in range(100):
    # 전파파
    y1 = affine1.forward(z)
    y2 = affine2.forward(y1)
    print(i, "모델 예측값 : ", np.round(y2, 3))
    loss = mse.forward(y2, t)
    print(i, "손실값 : ", np.round(loss, 3))

    # 역전파
    dout = np.ones(1,)
    dout = mse.backward(dout)
    dout = affine2.backward(dout)
    dout = affine1.backward(dout)

    # 경사하강법
    affine1.W = affine1.W - lr * affine1.dW
    affine1.b = affine1.b - lr * affine1.db
    affine2.W = affine2.W - lr * affine2.dW
    affine2.b = affine2.b - lr * affine2.db
    '''
    print("W1 : ", affine1.W)
    print("b1 : ", affine1.b)
    print("W2 : ", affine2.W)
    print("b2 : ", affine2.b)
    print("손실값 : ", loss)
    '''

0 모델 예측값 :  [[1.]]
0 손실값 :  0.0
1 모델 예측값 :  [[1.]]
1 손실값 :  0.0
2 모델 예측값 :  [[1.]]
2 손실값 :  0.0
3 모델 예측값 :  [[1.]]
3 손실값 :  0.0
4 모델 예측값 :  [[1.]]
4 손실값 :  0.0
5 모델 예측값 :  [[1.]]
5 손실값 :  0.0
6 모델 예측값 :  [[1.]]
6 손실값 :  0.0
7 모델 예측값 :  [[1.]]
7 손실값 :  0.0
8 모델 예측값 :  [[1.]]
8 손실값 :  0.0
9 모델 예측값 :  [[1.]]
9 손실값 :  0.0
10 모델 예측값 :  [[1.]]
10 손실값 :  0.0
11 모델 예측값 :  [[1.]]
11 손실값 :  0.0
12 모델 예측값 :  [[1.]]
12 손실값 :  0.0
13 모델 예측값 :  [[1.]]
13 손실값 :  0.0
14 모델 예측값 :  [[1.]]
14 손실값 :  0.0
15 모델 예측값 :  [[1.]]
15 손실값 :  0.0
16 모델 예측값 :  [[1.]]
16 손실값 :  0.0
17 모델 예측값 :  [[1.]]
17 손실값 :  0.0
18 모델 예측값 :  [[1.]]
18 손실값 :  0.0
19 모델 예측값 :  [[1.]]
19 손실값 :  0.0
20 모델 예측값 :  [[1.]]
20 손실값 :  0.0
21 모델 예측값 :  [[1.]]
21 손실값 :  0.0
22 모델 예측값 :  [[1.]]
22 손실값 :  0.0
23 모델 예측값 :  [[1.]]
23 손실값 :  0.0
24 모델 예측값 :  [[1.]]
24 손실값 :  0.0
25 모델 예측값 :  [[1.]]
25 손실값 :  0.0
26 모델 예측값 :  [[1.]]
26 손실값 :  0.0
27 모델 예측값 :  [[1.]]
27 손실값 :  0.0
28 모델 예측값 :  [[1.]]
28 손실값 :  0.0
29 모델 예측값 :  [[1.]]
29 손실값 :  0.0
